In [1]:
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup
import pandas as pd
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

GAME = ('Red_and_Blue', 'g1', 'gen1')
STARTERS = ['Bulbasaur','Charmander','Squirtle']
EEVEELUTION = 'Vaporeon'
AVAILABLE_POKEMON = []
PART_NUM = 14

types = ['Grass','Poison']

async def get_html(url, selector, sleep=5, retries=5):
    html = None
    for i in range(1, retries + 1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.webkit.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

html = await get_html(f"https://pokemondb.net/evolution#evo-{GAME[1]}", '#main')
soup = BeautifulSoup(html)
rows = soup.find_all('div', {'class':'infocard-filter-block'})
EVO_CHART = {}
for row in rows:
    names = row.find_all('a', {'class':'ent-name'})
    base = names[0].text.strip()
    if base == 'Eevee':
        EVO_CHART[base] = [EEVEELUTION]
    else:
        for i, name in enumerate(names):
            if i == 0:
                EVO_CHART[base] = []
            else:
                EVO_CHART[base].append(name.text.strip())

html = await get_html(f"https://bulbapedia.bulbagarden.net/wiki/Appendix:{GAME[0]}_walkthrough", '#bodyContent')
soup = BeautifulSoup(html)
parts = soup.find('table', {'class':'roundy'})
rows = parts.find_all('tr')
hrefs = []
for row in rows[1:]:
    part = row.find_all('th')[0]
    try:
        link = part.find('a')
        href = link.get('href')
        hrefs.append(href)
    except:
        continue

for href in hrefs[:PART_NUM]:
    LATEST_TEAM = []
    html = await get_html(f"https://bulbapedia.bulbagarden.net{href}", '.mw-parser-output')
    soup = BeautifulSoup(html)
    pokemon_tables = pd.read_html(str(soup), match='A colored background')
    for table in pokemon_tables:
        if isinstance(table.columns, pd.MultiIndex):
            table.columns = table.columns.get_level_values(0)
        if len(table) > 0:
            for pokemon in table['Pokémon']:
                if (str(pokemon) != 'nan' and not str(pokemon).startswith('A colored background') and
                    str(pokemon) not in STARTERS and 'Pokémon' not in str(pokemon) and pokemon not in AVAILABLE_POKEMON):
                        AVAILABLE_POKEMON.append(pokemon)
                        if pokemon in EVO_CHART:
                            print(f"NEW {pokemon}")
                            LATEST_TEAM.append(pokemon)
                        else:
                            found = False
                            for base, evolutions in EVO_CHART.items():
                                if pokemon == base or pokemon in evolutions:
                                    found = True
                            if not found:
                                print(f"NEW {pokemon} (DOES NOT EVOLVE)")
                                LATEST_TEAM.append(pokemon)

AVAILABLE_POKEMON = list(set(AVAILABLE_POKEMON))
TEAM = []
TEAM.append(EVO_CHART[STARTERS[0]][-1])

html = await get_html(f"https://pokemondb.net/pokedex/stats/{GAME[2]}", '#main')
soup = BeautifulSoup(html)
stats_table = pd.read_html(str(soup), attrs={'id':'pokedex'})[0].sort_values(by=['Total','#'], ascending=False, ignore_index=True)

for i in range(len(AVAILABLE_POKEMON)):
    for base, evolutions in EVO_CHART.items():
        if AVAILABLE_POKEMON[i] == base or AVAILABLE_POKEMON[i] in evolutions:
            if evolutions[-1] not in list(stats_table['Name']):
                if len(evolutions) > 1: 
                    if evolutions[-2] in list(stats_table['Name']):
                        AVAILABLE_POKEMON[i] = evolutions[-2]
                        print(f"{evolutions[-1]} devolved to {evolutions[-2]}")
                    else:
                        if len(evolutions) > 2:
                            AVAILABLE_POKEMON[i] = evolutions[-3]
                            print(f"{evolutions[-1]} devolved to {evolutions[-3]}")
            else:
                AVAILABLE_POKEMON[i] = evolutions[-1]

for index, row in stats_table.iterrows():
    type_used = False
    pok_types = row['Type'].split()
    for typ in pok_types:
        if typ in types:
            type_used = True
    if row['Name'] in AVAILABLE_POKEMON and not type_used:
        TEAM.append(row['Name'])
        types.extend(pok_types)
    if row['Name'] in AVAILABLE_POKEMON and type_used:
        print(f"SKIPPED {row['Name']}, {pok_types} already used")

print()
for pokemon in TEAM[:6]:
    if pokemon in LATEST_TEAM:
        print(f"NEED TO CATCH: {pokemon}")
TEAM[:6]

Pokémon evolution charts | Pokémon Database
Walkthrough:Pokémon Red and Blue - Bulbapedia, the community-driven Pokémon encyclopedia
Walkthrough:Pokémon Red and Blue/Part 1 - Bulbapedia, the community-driven Pokémon encyclopedia
Walkthrough:Pokémon Red and Blue/Part 2 - Bulbapedia, the community-driven Pokémon encyclopedia
NEW Pidgey
NEW Rattata
NEW Nidoran♀
NEW Nidoran♂
NEW Spearow
NEW Caterpie
NEW Weedle
Walkthrough:Pokémon Red and Blue/Part 3 - Bulbapedia, the community-driven Pokémon encyclopedia
Walkthrough:Pokémon Red and Blue/Part 4 - Bulbapedia, the community-driven Pokémon encyclopedia
NEW Magikarp
NEW Zubat
NEW Geodude
NEW Paras
NEW Ekans
NEW Sandshrew
Walkthrough:Pokémon Red and Blue/Part 5 - Bulbapedia, the community-driven Pokémon encyclopedia
NEW Oddish
NEW Bellsprout
NEW Abra
NEW Meowth
NEW Mankey
Walkthrough:Pokémon Red and Blue/Part 6 - Bulbapedia, the community-driven Pokémon encyclopedia
NEW Farfetch'd
NEW Fishing (DOES NOT EVOLVE)
Walkthrough:Pokémon Red and Blue/Pa

['Venusaur', 'Zapdos', 'Arcanine', 'Snorlax', 'Lapras', 'Machamp']

In [2]:
for pok in AVAILABLE_POKEMON:
    if pok not in list(stats_table['Name']):
        print(pok)

Surfing
Fighting Dojo
Fishing
